In [1]:
# import pandas as pd
# from osgeo import gdal
# import netCDF4
# import numpy as np
# import os
# import openpyxl
from functions_UI_to_BW import *
# import geowombat as gw
# from geowombat.data import rgbn

# make everything a function

In [2]:
crop_ui = "Maize (corn)"
lat_ui = 46.81228 
lon_ui = -71.21454

file_path_map = "../data/Mapping_data_Calcey.xlsx"
sheet_name_pest_map ="Mapping_pesticides"
folder_path_pest_background = "../data/NC_background_pest/"

In [5]:
crop = crop_proxy_excelwithsheet(crop=crop_ui, file_path=file_path_map, sheet_name=sheet_name_pest_map)
df_pest = get_pest_background_data(input_lat=lat_ui, input_lon=lon_ui, crop=crop, path_to_folder=folder_path_pest_background)
df_pest

,pesticide_name,local_low_value,local_high_value
0,"2,4-D",0.053369,0.053467
1,ACETOCHLOR,0.055025,0.065590
2,ALACHLOR,-1.000000,-1.000000
3,ATRAZINE,0.077804,0.293052
4,CHLORPYRIFOS,0.029396,0.059853
5,CLOPYRALID,0.001465,0.003314
6,DICAMBA,0.006988,0.027782
7,DIMETHENAMID-P,0.000676,0.012542
8,EPTC,0.000140,0.000140
9,GLUFOSINATE,0.003862,0.006710
